# WNBA Data Scraping

We are going to get player data from the website [stats.wnba.com](https://stats.wnba.com).



In [ ]:
from selenium import webdriver
import pandas as pd
import os

teams = {
    'Atlanta Dream':1611661330,
    'Chicago Sky':1611661329,
    'Connecticut Sun':1611661323,
    'Indiana Fever':1611661325,
    'New York Liberty':1611661313,
    'Washington Mystics':1611661322,
    'Dallas Wings':1611661321,
    'Las Vegas Aces':1611661319,
    'Los Angeles Sparks':1611661320,
    'Minnesota Lynx':1611661324,
    'Phoenix Mercury':1611661317,
    'Seattle Storm':1611661328 }

old_teams = {
    'Cleveland Rockers':1611661315,
    'Charlotte Sting':1611661314,
    'Houston Comets':1611661316,
    'Sacramento Monarchs':1611661318,
    'Miami Sol':1611661326,
    'Portland Fire':1611661327}

all_teams = {**teams, **old_teams}

#url = 'https://stats.wnba.com/team/1611661319/players-scoring/?Season=2023'

driver = webdriver.Chrome()

#season = 2023
for season in range(1997, 2024):
    #for team, id in teams.items():
    #for team, id in old_teams.items():
    for team, id in all_teams.items():
        # check if file exists
        if os.path.isfile(f'data/{team} - {season}.csv'):
            pass
        else:
            print(season, team)
            try:
                url = f'https://stats.wnba.com/team/{id}/players-scoring/?Season={season}'
                driver.get(url)
                tables = driver.find_elements('xpath', "//table")
                #df = pd.read_html(tables[3].get_attribute('outerHTML'))[0]
                for table in tables:
                    df = pd.read_html(table.get_attribute('outerHTML'))[0]
                    if df.columns[0] == 'Players' and df.columns[1] == 'GP':
                        df['TEAM'] = team
                        df['SEASON'] = season
                        df.to_csv(f'data/{team} - {season}.csv', index=False)
                        break
            except:
                print('Error')

driver.quit()

In [ ]:
# join all the csv files into one dataframe
import os
import pandas as pd
df_all = pd.DataFrame()
for file in os.listdir('data'):
    if file.endswith('.csv'):
        df = pd.read_csv(f'data/{file}')
        df_all = pd.concat([df_all, df])
# rename the Players column
df_all.rename(columns={'Players':'PLAYER'}, inplace=True)
# add a column for Team ID
df_all['TEAM_ID'] = df_all['TEAM'].map(all_teams)
df_all.to_csv('data/wnba.csv', index=False)